# Analysis of Campaign Effectiveness
1. Compare the effectiveness of various campaigns in terms of lead generation and conversion rate.
2. Evaluate the effectiveness of different marketing sources in generating quality leads.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px

In [ ]:
deals = pd.read_pickle('deals_df.pkl')
spend = pd.read_pickle('spend_df.pkl')

In [ ]:
deals.info()

# 1. Analysis of campaigns from the perspective of lead generation and conversion rate.

## Top-10 Campaigns: Lead Generation and Conversion

In [ ]:
# Data preparation: grouping deals by campaigns
campaign_data = deals[deals['Source'] != 'Offline'].groupby('Campaign').agg(
total_leads=('Id', 'count'), # Total number of leads
completed_leads=('Stage', lambda x: (x == 'Payment Done').sum()), # Completed leads
failed_leads=('Stage', lambda x: (x == 'Lost').sum()) # Lost leads
).reset_index()
len(campaign_data)

#### Since there are 154 advertising campaigns, I will leave the top 10 for analysis. All others will be grouped into 'Others.' According to the results, the top 10 included advertising campaigns that attracted more than 478 clients.

In [ ]:
# Adding click data from the spend table
clicks_info = spend.groupby('Campaign', observed=False)['Clicks'].sum().reset_index()
combined_data = pd.merge(campaign_data, clicks_info, on='Campaign', how='left')

# Excluding 'Unknown'
cleaned_data = combined_data[combined_data['Campaign'] != 'Unknown'].sort_values('total_leads', ascending=False)

# Selecting the top 10 campaigns
top_campaigns = cleaned_data.head(10)

# Summing the remaining campaigns into the "Other Campaigns" category
other_campaigns = cleaned_data.iloc[10:]
other_summary = pd.DataFrame({
    'Campaign': ['Other Campaigns'],
    'total_leads': other_campaigns['total_leads'].sum(),
    'completed_leads': other_campaigns['completed_leads'].sum(),
    'failed_leads': other_campaigns['failed_leads'].sum(),
    'Clicks': other_campaigns['Clicks'].sum()
})

# Combining the top 10 and "Other Campaigns"
final_campaign_data = pd.concat([top_campaigns, other_summary], ignore_index=True)

# Calculating percentages of successful and failed leads
final_campaign_data['Completed_leads(%)'] = (final_campaign_data['completed_leads'] / final_campaign_data['total_leads'] * 100).round(2)
final_campaign_data['Failed_leads(%)'] = (final_campaign_data['failed_leads'] / final_campaign_data['total_leads'] * 100).round(2)

plt.figure(figsize=(10, 8))
ax = plt.gca()

# Columns for total number of leads
bars = ax.bar(final_campaign_data['Campaign'], final_campaign_data['total_leads'], color='brown', label='Total number of leads')

# Lines for completed and lost leads
ax.plot(final_campaign_data['Campaign'], final_campaign_data['completed_leads'], color='forestgreen', marker='o', linestyle='-', label='Completed leads')
ax.plot(final_campaign_data['Campaign'], final_campaign_data['failed_leads'], color='coral', marker='o', linestyle='-.', label='Lost leads')

# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}', ha='center', va='bottom', fontsize=9)

# Setting up the chart
ax.set_title('Top-10 Campaigns: Lead Generation and Conversion')
ax.set_xlabel('Campaign Name')
ax.set_ylabel('Number of Leads')
ax.tick_params(axis='x', rotation=90)
ax.legend(loc='best')

plt.tight_layout()
plt.show()

### Chart Analysis: Top-10 Campaigns for Lead Generation and Conversion

1.**Total Number of Deals**
-**Leading Campaigns**:
-**`wide_DE`**and**`performancemax_digitalmarkt_ru_DE`**rank first and second respectively, each exceeding**2600 deals**.
- Other campaigns have significantly lower transaction volumes, ranging from**479 to 2653 deals**.

2.**Lost Deals**
-**High Loss Rate**:
- Campaigns**`wide_DE`**,**`performancemax_digitalmarkt_ru_DE`**and**`youtube_shorts_DE`**have the highest percentage of lost deals.
- Lost deals are visualized by a**red dashed line**, which shows the highest peaks for these campaigns.
-**Conclusion**:
- Although these campaigns generate a significant number of leads, a large portion of them**do not convert into successful deals**.

3.**Completed Stages**
-**Low Level of Completed Deals**:
- The**green dots**on the chart, representing the number of successful deals completed, are significantly lower compared to the total number of deals.
- This is particularly noticeable in the campaigns**`webinar`**,**`wide_webinar_DE`**and**`youtube_shorts_DE`**, where conversion rates are**less than 1%**.

4.**Comparative Analysis**
-**Campaigns with High Transaction Volume**:
-**`wide_DE`**,**`performancemax_digitalmarkt_ru_DE`**and**`youtube_shorts_DE`**show high loss rates.
- This indicates that, despite successful lead acquisition, the conversion process requires**significant improvement**.
-**Campaigns with Lower Transaction Volume**:
-**`women`**,**`Dis_DE`**and**`LAL_DE`**demonstrate a more balanced ratio of leads to successful conversions.
- This may indicate**more effective lead management**in these campaigns.

5.**Conclusions**
-**Need for Improvements**:
- High activity in campaigns such as**`wide_DE`**and**`performancemax_digitalmarkt_ru_DE`**is coupled with a high percentage of lost transactions.
- This indicates a need to revisit the approach to handling leads to**reduce losses and increase conversion efficiency**.
-**Potential Strategies**:
- Optimize the lead handling process in high-volume transaction campaigns.
- Focus on campaigns with low levels of completed deals to improve their efficiency.

## Graph for studying the conversion rate 
This graph will complement the analysis by showing not only the total number of leads but also their quality (conversion into successful deals).

In [ ]:
# Grouping data by sources
source_data = deals[deals['Campaign'] != 'Unknown'].groupby('Campaign', observed=True).agg(
total_leads=('Id', 'count'),
completed_leads=('Stage', lambda x: (x == 'Payment Done').sum())).reset_index()

# Calculating conversion percentage
source_data['Completed_leads(%)'] = (source_data['completed_leads'] / source_data['total_leads']) * 100

# Sorting by total number of leads in descending order
source_data = source_data.sort_values('total_leads', ascending=False)

# Selecting the top 10 most successful campaigns
top_10_campaigns = source_data.head(10)

# Building the chart
plt.figure(figsize=(10, 8))
ax = plt.gca()

# Bars for total number of leads
ax.bar(top_10_campaigns['Campaign'], top_10_campaigns['total_leads'], color='brown', label='Total Leads')
# Bars for successful leads (overlay)
ax.bar(top_10_campaigns['Campaign'], top_10_campaigns['completed_leads'], color='orange', label='Completed Leads')

# Line for conversion percentage
ax2 = ax.twinx()
ax2.plot(top_10_campaigns['Campaign'], top_10_campaigns['Completed_leads(%)'], color='blue', marker='o', linestyle='-.', label='Conversion Percentage')

# Adding numbers above the bars
for i, v in enumerate(top_10_campaigns['total_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center')
for i, v in enumerate(top_10_campaigns['completed_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center', color='white')

# Setting up the chart
ax.set_title('Top-10 Campaigns by Lead Generation and Conversion')
ax.set_xlabel('Campaign')
ax.set_ylabel('Number of Leads')
ax.tick_params(axis='x', rotation=45)
ax.legend(loc='center left', bbox_to_anchor=(0.5, 0.95), ncol=1)

# ax2.set_ylabel('Conversion Percentage (%)')
ax2.legend(loc='center right', bbox_to_anchor=(0.5, 0.95), ncol=3)

# Optimization and display
plt.tight_layout()
plt.show()
# Output the top 3 highest values for successful leads
print("Top-3 campaigns by number of completed leads:")
print(top_10_campaigns.nlargest(3, 'completed_leads'))

# Output the top 3 campaigns by conversion
print("Top-3 campaigns by conversion percentage:")
print(top_10_campaigns.nlargest(3, 'Completed_leads(%)'))

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.gca()

# Bars for the percentage of successful leads
bars = ax.bar(final_campaign_data['Campaign'], final_campaign_data['Completed_leads(%)'], color='#B30000', label='Percentage of Successful Leads')

# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

# Chart configuration
ax.set_title('Top 10 Campaigns: Conversion Rate')
ax.set_xlabel('Campaign Name')
ax.set_ylabel('Percentage of Successful Leads (%)')
ax.tick_params(axis='x', rotation=90)
# ax.legend(loc='best')

plt.tight_layout()
plt.show()

In [ ]:
# Creation of a dataframe with the number of leads by Campaign
campaign_leads = deals[deals['Campaign'] != 'Unknown'].groupby('Campaign', observed=True).agg(
total_leads=('Id', 'count')) .reset_index()

# Selection of the top-10 campaigns by total number of leads
top_10_campaigns = campaign_leads.sort_values('total_leads', ascending=False).head(10)['Campaign']

# Filtering deals by top-10 campaigns and excluding 'Unknown' quality
filtered_deals = deals[(deals['Campaign'].isin(top_10_campaigns)) & (deals['Quality'] != 'Unknown')]

# Grouping data by Quality and Campaign for Sunburst
multi_category_analysis = filtered_deals.groupby(['Quality', 'Campaign'], observed=True).size().reset_index(name='Count')

# Counting the number of deals with quality 'A - High' and 'C - Low' for each Source
a_high_counts = multi_category_analysis[multi_category_analysis['Quality'] == 'A - High'].set_index('Campaign')['Count']
c_low_counts = multi_category_analysis[multi_category_analysis['Quality'] == 'C - Low'].set_index('Campaign')['Count']

# Calculating the ratio for each Source
a_high_to_c_low_ratios = a_high_counts / c_low_counts

# Finding the leader by maximum ratio
max_ratio_leader = a_high_to_c_low_ratios.idxmax()
max_ratio = a_high_to_c_low_ratios.max()
max_a_high_count = a_high_counts[max_ratio_leader]
max_c_low_count = c_low_counts[max_ratio_leader]

# Finding the leader by minimum ratio
min_ratio_leader = a_high_to_c_low_ratios.idxmin()
min_ratio = a_high_to_c_low_ratios.min()
min_a_high_count = a_high_counts[min_ratio_leader]
min_c_low_count = c_low_counts[min_ratio_leader]

print(f"Leader by maximum A/C ratio: {max_ratio_leader} ({max_ratio:.2f}) [A-High: {max_a_high_count}, C-Low: {max_c_low_count}]")
print(f"Leader by minimum A/C ratio: {min_ratio_leader} ({min_ratio:.2f}) [A-High: {min_a_high_count}, C-Low: {min_c_low_count}]")

# Building a stacked bar chart
fig = px.bar(
multi_category_analysis,
x='Campaign',
y='Count',
color='Quality',
barmode='stack',  # Stacked chart
title='Distribution of deals by quality and top-10 campaigns',
labels={'Count': 'Number of deals', 'Campaign': 'Campaign', 'Quality': 'Quality'}
)

# Customizing the appearance
fig.update_layout(
xaxis_title="Campaign",
yaxis_title="Number of deals",
xaxis_tickangle=45,
height=500,
width=800,
legend_title="Quality"
)

fig.show()

## Analysis of the Temporal Dynamics of Campaigns
-**Goal**: To assess how the effectiveness of campaigns has changed over time (for example, by months or quarters)  
-**Type of Chart**: Heatmap  
-**What to Look For**: This will help identify seasonal trends or successful periods for specific campaigns.  

In [ ]:
# Filtering successful leads, excluding 'Unknown' and 'Offline', extracting month and year
successful_data = (deals[deals['Stage'] == 'Payment Done']
                   .query("Campaign != 'Unknown' and Source != 'Offline'")
                   .assign(**{'Month-Year': lambda df: df['Created Time'].dt.to_period('M')}))

# Selecting the top 10 campaigns by the number of successful leads and filtering the data
top_campaigns = successful_data['Campaign'].value_counts().head(10).index
monthly_data = (successful_data[successful_data['Campaign'].isin(top_campaigns)]
                .groupby(['Month-Year', 'Campaign'])
                .size()
                .unstack(fill_value=0))

plt.figure(figsize=(12, 8))
ax = plt.gca()

# Creating a heatmap
sns.heatmap(monthly_data, annot=True, fmt='d', linewidths=0.5, cbar=True, cmap='YlOrRd', ax=ax)

# Setting up the chart
ax.set_title('Heatmap of Successful Leads by Top 10 Campaigns')
ax.set_xlabel('Campaign')
ax.set_ylabel('Month-Year')
ax.tick_params(axis='x', rotation=45)

# Optimizing and displaying
plt.tight_layout()
plt.show()

# 2. Analysis of the effectiveness of marketing sources (` Source `)

## Sources Analysis: Lead Generation and Conversion

In [ ]:
# Preparing data: grouping deals by sources, excluding 'Offline'
source_data = deals[deals['Source'] != 'Offline'].groupby('Source', observed=True).agg(
    total_leads=(('Id', 'count')),  # Total number of leads
    completed_leads=(('Stage', lambda x: (x == 'Payment Done').sum())),  # Completed leads
    failed_leads=(('Stage', lambda x: (x == 'Lost').sum()))  # Lost leads
).reset_index()

# Adding click data from the spend table
clicks_info = spend.groupby('Source', observed=True)['Clicks'].sum().reset_index()
combined_data = pd.merge(source_data, clicks_info, on='Source', how='left')

# Filtering and sorting, excluding 'Unknown'
cleaned_data = combined_data[combined_data['Source'] != 'Unknown'].sort_values('total_leads', ascending=False)

# Selecting the top 10 sources
top_sources = cleaned_data.head(10)

# Summing the remaining sources into the "Other Sources" category
other_sources = cleaned_data.iloc[10:]
other_summary = pd.DataFrame({
    'Source': ['Other Sources'],
    'total_leads': other_sources['total_leads'].sum(),
    'completed_leads': other_sources['completed_leads'].sum(),
    'failed_leads': other_sources['failed_leads'].sum(),
    'Clicks': other_sources['Clicks'].sum()
})

# Combining the top 10 and "Other Sources"
final_source_data = pd.concat([top_sources, other_summary], ignore_index=True)

# Calculating percentages of successful and lost leads
final_source_data['Completed_leads(%)'] = (final_source_data['completed_leads'] / final_source_data['total_leads'] * 100).round(2)
final_source_data['Failed_leads(%)'] = (final_source_data['failed_leads'] / final_source_data['total_leads'] * 100).round(2)

# Plotting for sources
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Bars for total number of leads
bars = ax.bar(final_source_data['Source'], final_source_data['total_leads'], color='brown', label='Total Number of Leads')
# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f' {int(height)} ', ha='center', va='bottom', fontsize=9)

# Lines for completed and lost leads
ax.plot(final_source_data['Source'], final_source_data['completed_leads'], color='limegreen', marker='o', linestyle='-', label='Completed Leads')
ax.plot(final_source_data['Source'], final_source_data['failed_leads'], color='salmon', marker='o', linestyle='-.', label='Lost Leads')

# Setting up the chart
ax.set_title('Source Analysis: Lead Generation and Conversion')
ax.set_xlabel('Source')
ax.set_ylabel('Number of Leads')
ax.tick_params(axis='x', rotation=90)
ax.legend(loc='best')

# Optimizing and displaying
plt.tight_layout()
plt.show()

# Outputting the table with sources, total number of leads, number of completed leads, and percentage of completed leads
print("Top 3 sources by number of successful leads:")
print(final_source_data.nlargest(3, 'completed_leads'))

## Lead Distribution Chart by Sources
-**Goal**: To show which sources generate the most leads and compare them with the number of successful deals.  
-**Chart Type**: Stacked bar chart.  
-**Data**: Grouping by the ` Source ` column, counting the total number of leads and successful deals ( ` Stage == 'Payment Done' ` ).

In [ ]:
# Grouping data by sources
source_data = deals[deals['Source'] != 'Offline'].groupby('Source', observed=True).agg(
    total_leads=('Id', 'count'),
    completed_leads=('Stage', lambda x: (x == 'Payment Done').sum())
).reset_index()

# Calculating the conversion percentage
source_data['Completed_leads(%)'] = (source_data['completed_leads'] / source_data['total_leads']) * 100

# Sorting by total number of leads
source_data = source_data.sort_values('total_leads', ascending=False)

plt.figure(figsize=(10, 6))
ax = plt.gca()

# Bars for total number of leads
ax.bar(source_data['Source'], source_data['total_leads'], color='brown', label='Total Number of Leads')
# Bars for successful leads (overlay)
ax.bar(source_data['Source'], source_data['completed_leads'], color='orange', label='Completed Leads')

# Line for conversion percentage
ax2 = ax.twinx()
ax2.plot(source_data['Source'], source_data['Completed_leads(%)'], color='blue', marker='o', linestyle='-.', label='Conversion Percentage')

# Adding numbers above the bars
for i, v in enumerate(source_data['total_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center')
for i, v in enumerate(source_data['completed_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center', color='white')

# Setting up the chart
ax.set_title('Lead Distribution by Sources')
ax.set_xlabel('Source')
ax.set_ylabel('Number of Leads')
ax.tick_params(axis='x', rotation=45)
ax.legend(loc='center left', bbox_to_anchor=(0.5, 0.95), ncol=1)  # Legend positioned at the center

ax2.set_ylabel('Conversion Percentage (%)')
ax2.legend(loc='center right', bbox_to_anchor=(0.5, 0.95), ncol=3)
# Optimization and display
plt.tight_layout()
plt.show()

# Calculating the conversion percentage
source_data['Completed_leads(%)'] = (source_data['completed_leads'] / source_data['total_leads']) * 100

# Sorting by conversion percentage
source_data = source_data.sort_values('Completed_leads(%)', ascending=False)

# Outputting the top 3 sources by conversion percentage
print("Top 3 sources by conversion percentage:")
print(source_data.nlargest(3, 'Completed_leads(%)'))

In [ ]:
# Grouping data by sources and counting the number of leads
source_leads = deals.groupby('Source', observed=True).size().reset_index(name='Leads')

# Grouping data by sources and counting the number of successful deals
successful_deals_source = deals[deals['Stage'] == 'Payment Done'].groupby('Source', observed=True).size().reset_index(name='Successful Deals')

# Merging lead data and successful deals into one table
source_performance = pd.merge(source_leads, successful_deals_source, on='Source', how='left')

# Filling missing values with zeros if there are no successful deals
source_performance['Successful Deals'] = source_performance['Successful Deals'].fillna(0)

# Calculating the conversion rate for each source and rounding to 1 decimal place
source_performance['Conversion Rate'] = round((source_performance['Successful Deals'] / source_performance['Leads']) * 100, 1)

# Sorting by descending Conversion Rate
source_performance = source_performance.sort_values('Conversion Rate', ascending=False)

styled_performance = (source_performance.style
    .hide(axis='index')
    .format({
        'Leads': '{:,.0f}',
        'Successful Deals': '{:,.0f}',
        'Conversion Rate': '{:.1f}%'
    })
    .background_gradient(subset=['Conversion Rate'], cmap='YlOrRd')
    .set_properties(**{'text-align': 'center'})
    .set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ])
    .bar(subset=['Leads', 'Successful Deals'], color='#F4A261', align='mid')
)

display(styled_performance)

## Conversion Rate Chart by Sources
-**Goal**: Estimate the percentage of successful leads from each source.  
-**Chart Type**: Bar chart.

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Sorting data by descending percentage of successful leads
sorted_data = final_source_data.sort_values('Completed_leads(%)', ascending=False)

# Bars for the percentage of successful leads
bars = ax.bar(sorted_data['Source'], sorted_data['Completed_leads(%)'], color='#B30000', label='Percentage of Successful Leads')

# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

# Setting up the chart
ax.set_title('Conversion Rate by Source')
ax.set_xlabel('Source')
ax.set_ylabel('Percentage of Successful Leads (%)')
ax.tick_params(axis='x', rotation=45)
ax.legend(loc='best')

# Optimization and display
plt.tight_layout()
plt.show()

# Top 3 marketing sources by conversion rate
print("Top 3 marketing sources by conversion rate:")
top_conversion_sources = final_source_data.sort_values('Completed_leads(%)', ascending=False).head(3)
for _, row in top_conversion_sources.iterrows():
    print(f"{row['Source']}: {row['Completed_leads(%)']:.2f}% successful deals")

In [ ]:
deals['Quality'].unique()

# Comparison and Supplement of Analysis - Connection between Sources and Campaigns
-**Goal**: To check which sources are most often associated with successful campaigns.  
-**Type of Analysis**: Crosstabulation or heatmap.  
-**Data**: Use the columns `Source` and `Campaign`, count the number of successful deals for each combination.  
-**Benefit**: This will help understand which sources work best with specific campaigns.

In [ ]:
# Filtering successful leads (Stage == 'Payment Done' and Source != 'Offline')
successful_deals = deals[(deals['Stage'] == 'Payment Done') & (deals['Source'] != 'Offline')]

# Excluding campaign 'Unknown'
successful_deals = successful_deals[successful_deals['Campaign'] != 'Unknown']

# Selecting the top 10 campaigns by total number of successful leads
top_campaigns = successful_deals['Campaign'].value_counts().head(10).index

# Filtering data for only the top 10 campaigns
top_campaign_data = successful_deals[successful_deals['Campaign'].isin(top_campaigns)]

# Grouping by campaigns and sources, counting successful leads
campaign_source_data = top_campaign_data.groupby(['Campaign', 'Source'], observed=False).size().unstack(fill_value=0)

# Creating a mask to exclude the source 'Offline'
mask = campaign_source_data.columns != 'Offline'

plt.figure(figsize=(12, 8))
ax = plt.gca()

sns.heatmap(campaign_source_data.loc[:, mask], annot=True,
            cmap='YlOrRd',
            fmt='.1f',
            linewidths=0.5,
            cbar=True, ax=ax)

# Setting up the chart
ax.set_title('Heatmap of Successful Leads by Top 10 Campaigns and Channels')
ax.set_xlabel('Channel (Source)')
ax.set_ylabel('Campaign (Campaign)')
ax.tick_params(axis='x', rotation=45)
ax.tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

### Interpretation in the context of the heat map

-**Campaigns within sources:**  
- The heat map shows that not all campaigns within the same channel are equally effective. For example:
-**Facebook Ads**shows high results for the campaign**performancemax_digitalmarkt_ru_DE** (376 successful leads), but other campaigns have lower effectiveness.  
-**Tiktok Ads**demonstrates success for the campaign**12.07.2023wide_DE**(226 leads), indicating the potential for optimizing campaigns in this channel.

-**Focus on successful campaigns:**  
- The heat map allows identifying specific successful campaigns that should be scaled. For example,**performancemax_digitalmarkt_ru_DE** through Facebook Ads and **12.07.2023wide_DE** through Tiktok Ads.